# Normalize taxa list

Cleanup the normalized taxa list from the eODP researchers. Add PBDB taxa data.

In [1]:
import sys
sys.path.append('../../../')

import pandas as pd
import numpy as np
import requests

from config import CLEAN_DATA_DIR, OUTPUT_DIR, RAW_DATA_DIR

import scripts.normalize_taxa as nt
from scripts.normalize_data import remove_whitespace
import scripts.pbdb as pbdb

from scripts.pbdb import get_parent_taxa, PBDB_TAXA_NAME

In [2]:
date = '2022-04-28'

input_file = RAW_DATA_DIR/'PI_processed_files'/f'LIMS_Micropal_headers_PBDB_Taxonomy_notes_taxa_list_{date}.csv'

pi_taxa_file_pbdb = OUTPUT_DIR /'taxa'/'LIMS'/f'PI_normalized_taxa_list_with_pbdb_{date}.csv'
crosswalk_file = OUTPUT_DIR/'taxa'/'LIMS'/f"taxa_crosswalk_{date}.csv"
taxa_list_file = OUTPUT_DIR/'taxa'/'LIMS'/f"taxa_list_{date}.csv"

genus_pbdb_file = OUTPUT_DIR/'taxa'/'LIMS'/f"genera_pbdb_{date}.csv"
higher_taxa_pbdb_file = OUTPUT_DIR/'taxa'/'LIMS'/f"higher_taxa_pbdb_{date}.csv"



In [3]:
def log_df(df, row_count=5):
    print(df.shape)
    return df.head(row_count)


## fix incorect pbdb_taxon_id

incorported  pbdb_taxon_id that the PIs corrected into the taxalist.


In [23]:
df = pd.read_csv(input_file, skiprows = 9, dtype=str)
df = df.drop(df.index[[0, 1]])
df = df.dropna(how="all", axis="index")
df['corrected'] = False

log_df(df)

(4742, 33)


,taxon_group,verbatim_name,name,"name to use (if different from ""name"")",name comment field,Comment,Notes (change to Internal only notes?),Any taxon above genus,genus modifier,genus name,...,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,corrected
2,benthic_forams,Euuvigerina miozea (group) >100 m,Euuvigerina miozea (group) >100 m,Euuvigerina miozea,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",NaN,NaN,Euuvigerina,...,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,False
3,benthic_forams,Euuvigerina rodleyi (group) >50 m,Euuvigerina rodleyi (group) >50 m,Euuvigerina rodleyi,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",NaN,NaN,Euuvigerina,...,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,False
4,benthic_forams,Others,Others,Foraminifera indet.,NaN,not a taxa,Andy,Foraminifera indet.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,212476,Rhizaria,False
5,benthic_forams,Pleurostomellids comment,Pleurostomellids comment,Pleurostomellia indet.,NaN,not a taxa,Andy,Pleurostomellidae indet.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,False
6,benthic_forams,Ostracoda spp.,Ostracoda spp.,Ostracoda indet.,Ostracoda spp.,group abundance,"Summer 2020: go in as Ostracoda indet, and the...",Ostracoda indet.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,18891,Arthropoda,325038,Animalia,False


In [24]:
df.columns

Index(['taxon_group', 'verbatim_name', 'name',
       'name to use (if different from "name")', 'name comment field',
       'Comment', 'Notes (change to Internal only notes?)',
       'Any taxon above genus', 'genus modifier', 'genus name',
       'subgenera modifier', 'subgenera name', 'species modifier',
       'species name', 'subspecies modifier', 'subspecies name',
       'non-taxa descriptor', 'comments', 'pbdb_taxon_id', 'pbdb_taxon_name',
       'pbdb_taxon_rank', 'Corrections to pbdb_taxon_id', 'family_taxon_id',
       'family_taxon_name', 'order_taxon_id', 'order_taxon_name',
       'class_taxon_id', 'class_taxon_name', 'phylum_taxon_id',
       'phylum_taxon_name', 'kingdom_taxon_id', 'kingdom_taxon_name',
       'corrected'],
      dtype='object')

In [25]:
tmp = df[df['Corrections to pbdb_taxon_id'].notna() & (df['corrected'] == False)]
tmp['Corrections to pbdb_taxon_id'].unique()

array(['genus; id 1064', 'genus, id 1124', 'genus; taxon_no= 2092',
       'genus; ID 421517', 'genus; ID 432678', 'genus; ID 71247',
       'genus; ID 82145', 'genus; ID 432650', 'genus: ID 68421',
       'genus; ID 432651', 'genus; ID 443753', 'genus; ID 434997',
       'genus; ID 165526'], dtype=object)

In [26]:
pbdb.fix_pbdb_id(df, 'genus; id 1064', 1064)
pbdb.fix_pbdb_id(df, 'genus, id 1124', 1124)
pbdb.fix_pbdb_id(df, 'genus; taxon_no= 2092', 2092)
pbdb.fix_pbdb_id(df, 'genus; ID 421517', 421517)
pbdb.fix_pbdb_id(df, 'genus; ID 432678', 432678)
pbdb.fix_pbdb_id(df, 'genus; ID 71247', 71247)
pbdb.fix_pbdb_id(df, 'genus; ID 82145', 82145)
pbdb.fix_pbdb_id(df, 'genus; ID 432650', 432650)
pbdb.fix_pbdb_id(df, 'genus: ID 68421', 68421)
pbdb.fix_pbdb_id(df, 'genus; ID 432651', 432651)
pbdb.fix_pbdb_id(df, 'genus; ID 443753', 443753)
pbdb.fix_pbdb_id(df, 'genus; ID 434997', 434997)
pbdb.fix_pbdb_id(df, 'genus; ID 165526', 165526)


1064
1124
2092
421517
432678
71247
82145
432650
68421
432651
443753
434997
165526


In [27]:
df.shape

(4742, 35)

In [28]:
del df['corrected']

In [29]:
df.to_csv(pi_taxa_file_pbdb, index=False)

## Create crosswalk csv

In [33]:
df = pd.read_csv(pi_taxa_file_pbdb, dtype=str)
log_df(df, 3)
# 4742

(4742, 34)


,taxon_group,verbatim_name,name,"name to use (if different from ""name"")",name comment field,Comment,Notes (change to Internal only notes?),Any taxon above genus,genus modifier,genus name,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,benthic_forams,Euuvigerina miozea (group) >100 m,Euuvigerina miozea (group) >100 m,Euuvigerina miozea,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",NaN,NaN,Euuvigerina,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
1,benthic_forams,Euuvigerina rodleyi (group) >50 m,Euuvigerina rodleyi (group) >50 m,Euuvigerina rodleyi,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",NaN,NaN,Euuvigerina,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
2,benthic_forams,Others,Others,Foraminifera indet.,NaN,not a taxa,Andy,Foraminifera indet.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,212476,Rhizaria,NaN,NaN


In [34]:
df.columns

Index(['taxon_group', 'verbatim_name', 'name',
       'name to use (if different from "name")', 'name comment field',
       'Comment', 'Notes (change to Internal only notes?)',
       'Any taxon above genus', 'genus modifier', 'genus name',
       'subgenera modifier', 'subgenera name', 'species modifier',
       'species name', 'subspecies modifier', 'subspecies name',
       'non-taxa descriptor', 'comments', 'pbdb_taxon_id', 'pbdb_taxon_name',
       'pbdb_taxon_rank', 'Corrections to pbdb_taxon_id', 'family_taxon_id',
       'family_taxon_name', 'order_taxon_id', 'order_taxon_name',
       'class_taxon_id', 'class_taxon_name', 'phylum_taxon_id',
       'phylum_taxon_name', 'kingdom_taxon_id', 'kingdom_taxon_name',
       'unranked clade_taxon_id', 'unranked clade_taxon_name'],
      dtype='object')

In [35]:
fields = nt.taxa_rank_fields + nt.taxa_fields + nt.metadata_fields  
fields

['Any taxon above genus',
 'genus modifier',
 'genus name',
 'subgenera modifier',
 'subgenera name',
 'species modifier',
 'species name',
 'subspecies modifier',
 'subspecies name',
 'non-taxa descriptor',
 'normalized_name',
 'taxon_group',
 'verbatim_name',
 'name comment field',
 'Comment',
 'Notes (change to Internal only notes?)',
 'comments']

In [36]:
filtered_taxa = pd.DataFrame(df, columns=fields)
remove_whitespace(filtered_taxa)

log_df(filtered_taxa, 2)


(4742, 17)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,name comment field,Comment,Notes (change to Internal only notes?),comments
0,,,Euuvigerina,,,,miozea,,,,NaN,benthic_forams,Euuvigerina miozea (group) >100 m,,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",>100 m group
1,,,Euuvigerina,,,,rodleyi,,,,NaN,benthic_forams,Euuvigerina rodleyi (group) >50 m,,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",>50 m group


set normalized_name using the taxa fields

In [37]:
nt.add_normalized_name_column(filtered_taxa)
log_df(filtered_taxa)

filtered_taxa = filtered_taxa.drop(filtered_taxa[filtered_taxa['normalized_name'] == ''].index)
log_df(filtered_taxa)

filtered_taxa.drop_duplicates(keep='first', inplace=True, subset =['verbatim_name', 'normalized_name'])
log_df(filtered_taxa)
# 4588

(4742, 17)
(4733, 17)
(4588, 17)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,name comment field,Comment,Notes (change to Internal only notes?),comments
0,,,Euuvigerina,,,,miozea,,,,Euuvigerina miozea,benthic_forams,Euuvigerina miozea (group) >100 m,,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",>100 m group
1,,,Euuvigerina,,,,rodleyi,,,,Euuvigerina rodleyi,benthic_forams,Euuvigerina rodleyi (group) >50 m,,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",>50 m group
2,Foraminifera indet.,,,,,,,,,,Foraminifera indet.,benthic_forams,Others,,not a taxa,Andy,other benthic foraminifera
3,Pleurostomellidae indet.,,,,,,,,,,Pleurostomellidae indet.,benthic_forams,Pleurostomellids comment,,not a taxa,Andy,
4,Ostracoda indet.,,,,,,,,,,Ostracoda indet.,benthic_forams,Ostracoda spp.,Ostracoda spp.,group abundance,"Summer 2020: go in as Ostracoda indet, and the...",


create crosswalk csv

In [38]:
filtered_taxa.to_csv(crosswalk_file, index=False)

## Create taxa list csv

create taxa list with unique taxa fields

In [39]:
df = pd.read_csv(pi_taxa_file_pbdb, dtype=str)
log_df(df, 3)

(4742, 34)


,taxon_group,verbatim_name,name,"name to use (if different from ""name"")",name comment field,Comment,Notes (change to Internal only notes?),Any taxon above genus,genus modifier,genus name,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,benthic_forams,Euuvigerina miozea (group) >100 m,Euuvigerina miozea (group) >100 m,Euuvigerina miozea,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",NaN,NaN,Euuvigerina,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
1,benthic_forams,Euuvigerina rodleyi (group) >50 m,Euuvigerina rodleyi (group) >50 m,Euuvigerina rodleyi,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",NaN,NaN,Euuvigerina,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
2,benthic_forams,Others,Others,Foraminifera indet.,NaN,not a taxa,Andy,Foraminifera indet.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,212476,Rhizaria,NaN,NaN


In [40]:
fields = nt.taxa_rank_fields  + nt.taxa_fields + nt.pdbd_fields 
fields

['Any taxon above genus',
 'genus modifier',
 'genus name',
 'subgenera modifier',
 'subgenera name',
 'species modifier',
 'species name',
 'subspecies modifier',
 'subspecies name',
 'non-taxa descriptor',
 'normalized_name',
 'taxon_group',
 'pbdb_taxon_id',
 'pbdb_taxon_name',
 'pbdb_taxon_rank',
 'family_taxon_id',
 'family_taxon_name',
 'order_taxon_id',
 'order_taxon_name',
 'class_taxon_id',
 'class_taxon_name',
 'phylum_taxon_id',
 'phylum_taxon_name',
 'kingdom_taxon_id',
 'kingdom_taxon_name']

In [41]:
filtered_taxa = pd.DataFrame(df, columns=fields)
remove_whitespace(filtered_taxa)

log_df(filtered_taxa, 2)

(4742, 25)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,,,Euuvigerina,,,,miozea,,,,...,,,,,,,288974,Foraminifera,212476,Rhizaria
1,,,Euuvigerina,,,,rodleyi,,,,...,,,,,,,288974,Foraminifera,212476,Rhizaria


In [42]:
nt.add_normalized_name_column(filtered_taxa)
log_df(filtered_taxa)

filtered_taxa = filtered_taxa.drop(filtered_taxa[filtered_taxa['normalized_name'] == ''].index)
log_df(filtered_taxa)

filtered_taxa.drop_duplicates(keep='first', inplace=True, subset =[ 'normalized_name', 'taxon_group'])
log_df(filtered_taxa)

(4742, 25)
(4733, 25)
(4172, 25)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,,,Euuvigerina,,,,miozea,,,,...,,,,,,,288974,Foraminifera,212476,Rhizaria
1,,,Euuvigerina,,,,rodleyi,,,,...,,,,,,,288974,Foraminifera,212476,Rhizaria
2,Foraminifera indet.,,,,,,,,,,...,,,,,,,,,212476,Rhizaria
3,Pleurostomellidae indet.,,,,,,,,,,...,,,,,,,288974,Foraminifera,212476,Rhizaria
4,Ostracoda indet.,,,,,,,,,,...,,,,,,,18891,Arthropoda,325038,Animalia


In [43]:
taxa_df = pd.DataFrame(filtered_taxa, columns=fields)
taxa_df = taxa_df.drop_duplicates() 

log_df(taxa_df)
# 4172

(4172, 25)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,,,Euuvigerina,,,,miozea,,,,...,,,,,,,288974,Foraminifera,212476,Rhizaria
1,,,Euuvigerina,,,,rodleyi,,,,...,,,,,,,288974,Foraminifera,212476,Rhizaria
2,Foraminifera indet.,,,,,,,,,,...,,,,,,,,,212476,Rhizaria
3,Pleurostomellidae indet.,,,,,,,,,,...,,,,,,,288974,Foraminifera,212476,Rhizaria
4,Ostracoda indet.,,,,,,,,,,...,,,,,,,18891,Arthropoda,325038,Animalia


In [44]:
taxa_df.to_csv(taxa_list_file, index=False)

## create genera csv

In [45]:
taxa_df = pd.read_csv(taxa_list_file, dtype=str)
log_df(taxa_df)

(4204, 25)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,NaN,NaN,Euuvigerina,NaN,NaN,NaN,miozea,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
1,NaN,NaN,Euuvigerina,NaN,NaN,NaN,rodleyi,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
2,Foraminifera indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,212476,Rhizaria
3,Pleurostomellidae indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
4,Ostracoda indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,18891,Arthropoda,325038,Animalia


In [46]:
genus_df = pd.DataFrame(taxa_df['genus name'].str.strip().dropna().unique(), columns=['genus name']) 

log_df(genus_df)

(1015, 1)


,genus name
0,Euuvigerina
1,Nodosaria
2,Cibicides
3,Brizalina
4,Candeina


In [47]:
genus_df.loc[genus_df['genus name'] == 'Martinotiella', 'genus name'] = 'Martinottiella'

In [54]:
for index, row in genus_df.iterrows(): 
    if index % 50 == 0:
        print(index, end=' ')

    url =  PBDB_TAXA_NAME +  row['genus name']
        
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()["records"]
        if len(data) == 1:
            genus_df.at[index, f'pbdb_taxon_id'] = str(data[0]["taxon_no"])
            genus_df.at[index, f'pbdb_taxon_name'] = data[0]["taxon_name"]
            genus_df.at[index, f'pbdb_taxon_rank'] = data[0]["taxon_rank"]
            
            round = 0
            get_parent_taxa(genus_df, data[0]["parent_no"], data[0]["taxon_rank"], round, index, None)

                

0 50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 1000 

In [49]:
genus_df = genus_df.reindex(columns=[
    'genus name', 
    'pbdb_taxon_id', 'pbdb_taxon_name', 'pbdb_taxon_rank',
    'family_taxon_id', 'family_taxon_name',
    'order_taxon_id', 'order_taxon_name', 
    'class_taxon_id', 'class_taxon_name',
    'phylum_taxon_id', 'phylum_taxon_name', 
    'kingdom_taxon_id', 'kingdom_taxon_name', 
    ])

In [50]:
genus_df[0:20]

,genus name,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,Euuvigerina,1408,Euuvigerina,genus,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
1,Nodosaria,1952,Nodosaria,genus,82197,Nodosariidae,429322,Nodosariida,428875,Nodosariata,288974,Foraminifera,212476,Rhizaria
2,Cibicides,1107,Cibicides,genus,82208,Cibicididae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
3,Brizalina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Candeina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Dentoglobigerina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Globigerina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Globigerinella,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Globigerinita,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Globigerinoides,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
genus_df.shape

(1015, 14)

In [52]:
genus_df.to_csv(genus_pbdb_file, index=False)

### add genus pbdb to input_file

In [288]:
df = pd.read_csv(input_file, skiprows = 9, dtype=str)
log_df(df)


(4754, 21)


,taxon_group,verbatim_name,name,"name to use (if different from ""name"")",name comment field,Comment,notes,Any taxon above genus,genus modifier,genus name,...,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,comments,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank
0,NaN,RESOLVED NAMES BY TAXONOMIC GROUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,benthic_forams,Pyrite,Pyrite,NaN,NaN,"not a taxa; Going into Macrostrat, not PBDB",Summer 2020: deal with later; should go in as ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,benthic_forams,Euuvigerina miozea (group) >100 m,Euuvigerina miozea (group) >100 m,Euuvigerina miozea,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",NaN,NaN,Euuvigerina,...,NaN,NaN,miozea,NaN,NaN,NaN,>100 m group,NaN,NaN,NaN
3,benthic_forams,Euuvigerina rodleyi (group) >50 m,Euuvigerina rodleyi (group) >50 m,Euuvigerina rodleyi,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",NaN,NaN,Euuvigerina,...,NaN,NaN,rodleyi,NaN,NaN,NaN,>50 m group,NaN,NaN,NaN
4,benthic_forams,Others,Others,Foraminifera indet.,NaN,not a taxa,Andy,Foraminifera indet.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,other benthic foraminifera,NaN,NaN,NaN


In [269]:
genus_df= pd.read_csv(genus_pbdb_file, dtype=str)

log_df(genus_df)

(1015, 14)


,genus name,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,Euuvigerina,1408,Euuvigerina,genus,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
1,Nodosaria,1952,Nodosaria,genus,82197,Nodosariidae,429322,Nodosariida,428875,Nodosariata,288974,Foraminifera,212476,Rhizaria
2,Cibicides,1107,Cibicides,genus,82208,Cibicididae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
3,Brizalina,1017,Brizalina,genus,112279,Bolivinidae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
4,Candeina,1053,Candeina,genus,422277,Candeinidae,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria


In [289]:
df['genus name'] = df['genus name'].str.strip()
genus_df['genus name'] = genus_df['genus name'].str.strip()

merged_df = df.merge(genus_df, on="genus name", how="left", suffixes=('_prev', None))
log_df(merged_df)

(4754, 34)


,taxon_group,verbatim_name,name,"name to use (if different from ""name"")",name comment field,Comment,notes,Any taxon above genus,genus modifier,genus name,...,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,NaN,RESOLVED NAMES BY TAXONOMIC GROUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,benthic_forams,Pyrite,Pyrite,NaN,NaN,"not a taxa; Going into Macrostrat, not PBDB",Summer 2020: deal with later; should go in as ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,benthic_forams,Euuvigerina miozea (group) >100 m,Euuvigerina miozea (group) >100 m,Euuvigerina miozea,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",NaN,NaN,Euuvigerina,...,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
3,benthic_forams,Euuvigerina rodleyi (group) >50 m,Euuvigerina rodleyi (group) >50 m,Euuvigerina rodleyi,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",NaN,NaN,Euuvigerina,...,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
4,benthic_forams,Others,Others,Foraminifera indet.,NaN,not a taxa,Andy,Foraminifera indet.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
merged_df.to_csv(pi_taxa_file_pbdb, index=False)

## create higher csv

In [9]:
taxa_df = pd.read_csv(taxa_list_file)
log_df(taxa_df)

(4209, 12)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group
0,NaN,NaN,Euuvigerina,NaN,NaN,NaN,miozea,NaN,NaN,NaN,Euuvigerina miozea,benthic_forams
1,NaN,NaN,Euuvigerina,NaN,NaN,NaN,rodleyi,NaN,NaN,NaN,Euuvigerina rodleyi,benthic_forams
2,Foraminifera indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foraminifera indet.,benthic_forams
3,Pleurostomellidae indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pleurostomellidae indet.,benthic_forams
4,Ostracoda indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ostracoda indet.,benthic_forams


In [120]:
higher_df = pd.DataFrame(taxa_df['Any taxon above genus'].str.strip().dropna().unique(), columns=['Any taxon above genus']) 
higher_df.replace('', np.nan, inplace=True)
higher_df.dropna(inplace=True)
log_df(higher_df)

(66, 1)


,Any taxon above genus
0,Foraminifera indet.
1,Pleurostomellidae indet.
2,Ostracoda indet.
3,Textulariia indet.
4,Elphidiidae indet.


In [123]:
for index, row in higher_df.iterrows():        
    if index % 20 == 0:
        print(index)
        
    name = row['Any taxon above genus'].replace(' indet.', '')

  
    url =  PBDB_TAXA_NAME +  name
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()["records"]
        if len(data) == 1:
            higher_df.at[index, f'pbdb_taxon_id'] = str(data[0]["taxon_no"])
            higher_df.at[index, f'pbdb_taxon_name'] = data[0]["taxon_name"]
            higher_df.at[index, f'pbdb_taxon_rank'] = data[0]["taxon_rank"]
            
            round = 0
            get_parent_taxa(higher_df, data[0]["parent_no"], data[0]["taxon_rank"], round, index, None)
        else:
            print(name, len(data), ' found')
    else:
        print(name, ' not found')

0
"Challengeria spp."  not found
"Phytolith"  not found
20
"Amorphous organic matter"  not found
"Black phytoclasts"  not found
"Brown phytoclasts"  not found
"round browns, psilate"  not found
"Terrestrial palynomorphs"  not found
"Unknown taxa"  not found
"other pollen"  not found
"Spores"  not found
"Fungal spores"  not found
"Pollen/spores reworked"  not found
"Other pollen"  not found
40
"Sporomorphs"  not found
"Black woody phytoclasts"  not found
"Brown woody phytoclasts"  not found
"Saccate pollen"  not found
"Monolete ornamented"  not found
"Monolete psilate"  not found
"Trilete ornamented"  not found
"Trilete psilate"  not found
60


In [124]:
higher_df.columns

Index(['Any taxon above genus', 'pbdb_taxon_id', 'pbdb_taxon_name',
       'pbdb_taxon_rank', 'kingdom_taxon_id', 'kingdom_taxon_name',
       'phylum_taxon_id', 'phylum_taxon_name'],
      dtype='object')

In [ ]:
higher_df = higher_df.reindex(columns=[
    'Any taxon above genus', 
    'pbdb_taxon_id', 'pbdb_taxon_name', 'pbdb_taxon_rank',
    'order_taxon_id', 'order_taxon_name', 
    'class_taxon_id', 'class_taxon_name',
    'phylum_taxon_id', 'phylum_taxon_name', 
    'kingdom_taxon_id', 'kingdom_taxon_name', 
    ])
higher_df

In [68]:
higher_df.to_csv(higher_taxa_pbdb_file, index=False)

### add higher pbdb to input_file

In [125]:
df = pd.read_csv(pi_taxa_file_pbdb,  dtype=str)
log_df(df)

(4754, 34)


,taxon_group,verbatim_name,name,"name to use (if different from ""name"")",name comment field,Comment,notes,Any taxon above genus,genus modifier,genus name,...,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,NaN,RESOLVED NAMES BY TAXONOMIC GROUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,benthic_forams,Pyrite,Pyrite,NaN,NaN,"not a taxa; Going into Macrostrat, not PBDB",Summer 2020: deal with later; should go in as ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,benthic_forams,Euuvigerina miozea (group) >100 m,Euuvigerina miozea (group) >100 m,Euuvigerina miozea,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",NaN,NaN,Euuvigerina,...,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
3,benthic_forams,Euuvigerina rodleyi (group) >50 m,Euuvigerina rodleyi (group) >50 m,Euuvigerina rodleyi,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",NaN,NaN,Euuvigerina,...,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
4,benthic_forams,Others,Others,Foraminifera indet.,NaN,not a taxa,Andy,Foraminifera indet.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
df.columns

Index(['taxon_group', 'verbatim_name', 'name',
       'name to use (if different from "name")', 'name comment field',
       'Comment', 'notes', 'Any taxon above genus', 'genus modifier',
       'genus name', 'subgenera modifier', 'subgenera name',
       'species modifier', 'species name', 'subspecies modifier',
       'subspecies name', 'non-taxa descriptor', 'comments',
       'pbdb_taxon_id_prev', 'pbdb_taxon_name_prev', 'pbdb_taxon_rank_prev',
       'pbdb_taxon_id', 'pbdb_taxon_name', 'pbdb_taxon_rank',
       'family_taxon_id', 'family_taxon_name', 'order_taxon_id',
       'order_taxon_name', 'class_taxon_id', 'class_taxon_name',
       'phylum_taxon_id', 'phylum_taxon_name', 'kingdom_taxon_id',
       'kingdom_taxon_name'],
      dtype='object')

In [126]:
higher_df = pd.read_csv(higher_taxa_pbdb_file, dtype=str)

In [127]:
for index, row in higher_df.iterrows():    
    for col in higher_df.columns:
        if col == 'Any taxon above genus':
            continue

        df.loc[df['Any taxon above genus'] == row['Any taxon above genus'], col] = row[col]

    

In [129]:
df.to_csv(pi_taxa_file_pbdb,  index=False)
